In [6]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [7]:
import pandas as pd

df = pd.read_excel("../data/raw/emdat.xlsx")
df.head()

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,OFDA/BHA Response,Appeal,Declaration,AID Contribution ('000 US$),Magnitude,Magnitude Scale,Latitude,Longitude,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,GADM Admin Units,Entry Date,Last Update
0,2010-0416-IND,No,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,IND,India,Southern Asia,Asia,Lakhimpur district (Assam province),Monsoonal rain,NaN,No,No,No,NaN,12555.0,Km2,26.235,92.25,Singora,2010,9,5.0,2010,9,17.0,NaN,NaN,30000.0,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,69.513293,"[{""adm2_code"":70091,""adm2_name"":""Lakhimpur""}]","[{""gid_2"":""IND.4.20_1"",""migration_date"":""2025-...",2011-01-10,2025-12-20
1,2023-0846-IND,No,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,IND,India,Southern Asia,Asia,"Thoothukudi, Tirunelveli, Kanniyakumari and Te...",Heavy rains,NaN,No,No,No,NaN,NaN,Km2,NaN,NaN,NaN,2023,12,18.0,2023,12,20.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.134993,NaN,"[{""gid_2"":""IND.31.25_1"",""name_2"":""Thoothukkudi...",2023-12-21,2026-01-16
2,2018-0049-IND,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,IND,India,Southern Asia,Asia,New Delhi,NaN,Fog,No,No,No,NaN,4.2,°C,NaN,NaN,NaN,2018,1,1.0,2018,2,7.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.049596,"[{""adm2_code"":17633,""adm2_name"":""Delhi""}]","[{""gid_2"":""IND.25.1_1"",""migration_date"":""2025-...",2018-02-27,2025-12-20
3,2000-0444-IND,No,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,IND,India,Southern Asia,Asia,"East Siang, 70041, West Siang, 70054, Upper Si...",NaN,NaN,No,No,No,NaN,NaN,Km2,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.895152,"[{""adm2_code"":70041,""adm2_name"":""Administrativ...","[{""gid_2"":""IND.3.5_1"",""migration_date"":""2025-1...",2003-07-01,2025-12-20
4,2000-0332-IND,No,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,IND,India,Southern Asia,Asia,"Yingkiang,Tuting villages (Upper Siang (70051)...",NaN,Broken Dam/Burst bank,No,No,No,NaN,55000.0,Km2,NaN,NaN,NaN,2000,6,10.0,2000,6,10.0,20.0,NaN,3000000.0,NaN,3000000.0,NaN,NaN,NaN,NaN,NaN,NaN,54.895152,"[{""adm2_code"":17582,""adm2_name"":""Golaghat""},{""...","[{""gid_2"":""IND.3.11_1"",""migration_date"":""2025-...",2005-09-15,2025-12-20


In [8]:
df.shape
df.info()
df.columns

<class 'pandas.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 47 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   DisNo.                                     392 non-null    str    
 1   Historic                                   392 non-null    str    
 2   Classification Key                         392 non-null    str    
 3   Disaster Group                             392 non-null    str    
 4   Disaster Subgroup                          392 non-null    str    
 5   Disaster Type                              392 non-null    str    
 6   Disaster Subtype                           392 non-null    str    
 7   External IDs                               105 non-null    str    
 8   Event Name                                 36 non-null     str    
 9   ISO                                        392 non-null    str    
 10  Country                              

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'GADM Admin Units', 'Entry Date', 'Last Update'],
      dtype='str')

In [9]:
core_cols = [
    "Disaster Type",
    "Country",
    "Start Year",
    "Total Deaths",
    "No. Injured",
    "No. Affected",
    "No. Homeless",
    "Total Affected",
    "Total Damage ('000 US$)"
]

core_df = df[core_cols]
core_df.head()

,Disaster Type,Country,Start Year,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Total Damage ('000 US$)
0,Flood,India,2010,NaN,NaN,30000.0,NaN,30000.0,NaN
1,Flood,India,2023,10.0,NaN,NaN,NaN,NaN,NaN
2,Extreme temperature,India,2018,44.0,NaN,NaN,NaN,NaN,NaN
3,Flood,India,2000,130.0,NaN,NaN,NaN,NaN,NaN
4,Flood,India,2000,20.0,NaN,3000000.0,NaN,3000000.0,NaN


In [10]:
core_df.isnull().sum

<bound method DataFrame.sum of      Disaster Type  Country  ...  Total Affected  Total Damage ('000 US$)
0            False    False  ...           False                     True
1            False    False  ...            True                     True
2            False    False  ...            True                     True
3            False    False  ...            True                     True
4            False    False  ...           False                     True
..             ...      ...  ...             ...                      ...
387          False    False  ...           False                    False
388          False    False  ...           False                     True
389          False    False  ...            True                     True
390          False    False  ...           False                     True
391          False    False  ...           False                    False

[392 rows x 9 columns]>

In [11]:
from src.preprocess import load_and_clean_data

clean_df = load_and_clean_data("../data/raw/emdat.xlsx")
clean_df.info()
clean_df.head()

<class 'pandas.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Disaster Type            392 non-null    str    
 1   Country                  392 non-null    str    
 2   Start Year               392 non-null    int64  
 3   Total Deaths             392 non-null    float64
 4   No. Injured              392 non-null    float64
 5   No. Affected             392 non-null    float64
 6   No. Homeless             392 non-null    float64
 7   Total Affected           392 non-null    float64
 8   Total Damage ('000 US$)  392 non-null    float64
dtypes: float64(6), int64(1), str(2)
memory usage: 27.7 KB


,Disaster Type,Country,Start Year,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Total Damage ('000 US$)
0,Flood,India,2010,0.0,0.0,30000.0,0.0,30000.0,0.0
1,Flood,India,2023,10.0,0.0,0.0,0.0,0.0,0.0
2,Extreme temperature,India,2018,44.0,0.0,0.0,0.0,0.0,0.0
3,Flood,India,2000,130.0,0.0,0.0,0.0,0.0,0.0
4,Flood,India,2000,20.0,0.0,3000000.0,0.0,3000000.0,0.0


In [12]:
clean_df.head()

,Disaster Type,Country,Start Year,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Total Damage ('000 US$)
0,Flood,India,2010,0.0,0.0,30000.0,0.0,30000.0,0.0
1,Flood,India,2023,10.0,0.0,0.0,0.0,0.0,0.0
2,Extreme temperature,India,2018,44.0,0.0,0.0,0.0,0.0,0.0
3,Flood,India,2000,130.0,0.0,0.0,0.0,0.0,0.0
4,Flood,India,2000,20.0,0.0,3000000.0,0.0,3000000.0,0.0


In [13]:
from src.feature_engineering import create_priority_score

scored_df = create_priority_score(clean_df)
scored_df[["Disaster Type", "Country", "priority_score"]].sort_values(
    "priority_score", ascending=False
).head(10)


ImportError: cannot import name 'create_priority_score' from 'src.feature_engineering' (d:\disaster_relief_ml\src\feature_engineering.py)

In [15]:
from src.preprocess import load_and_clean_data
from src.feature_engineering import add_severity_score, add_severity_level

df = load_and_clean_data("../data/raw/emdat.xlsx")

df = add_severity_score(df)
df = add_severity_level(df)

df["severity_level"] = df["severity_level"].map({
    0: "Low",
    1: "Medium",
    2: "High"
})

df[["severity_score", "severity_level"]].head()

df["severity_level"].value_counts()

severity_level
High      133
Low       130
Medium    129
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [17]:
X = df.drop(columns=["severity_level", "severity_score"])
y = df["severity_level"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [16]:
df.columns

Index(['Disaster Type', 'Country', 'Start Year', 'Total Deaths', 'No. Injured',
       'No. Affected', 'No. Homeless', 'Total Affected',
       'Total Damage ('000 US$)', 'log_deaths', 'log_injured', 'log_affected',
       'log_damage', 'severity_score', 'severity_level'],
      dtype='str')

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


ValueError: could not convert string to float: 'Storm'

In [22]:
X_train.dtypes


Disaster Type                  str
Country                        str
Start Year                   int64
Total Deaths               float64
No. Injured                float64
No. Affected               float64
No. Homeless               float64
Total Affected             float64
Total Damage ('000 US$)    float64
log_deaths                 float64
log_injured                float64
log_affected               float64
log_damage                 float64
dtype: object

In [23]:
X = pd.get_dummies(X, drop_first=True)


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)


In [27]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)


d:\disaster_relief_ml\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [28]:
y_pred = model.predict(X_test)


In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        High       0.84      0.78      0.81        27
         Low       0.77      0.92      0.84        26
      Medium       0.65      0.58      0.61        26

    accuracy                           0.76        79
   macro avg       0.76      0.76      0.75        79
weighted avg       0.76      0.76      0.75        79



In [30]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[21,  0,  6],
       [ 0, 24,  2],
       [ 4,  7, 15]])